In [42]:
import pandas as pd
import numpy as np
import yfinance as yf
import warnings

class MomentumPortfolioBuilder:
    def __init__(self, stocks, portfolio_size=10):
        self.stocks = stocks
        self.portfolio_size = portfolio_size
        warnings.filterwarnings('ignore')  # Suppress warnings

    def fetch_stock_data(self, period='1y'):
        """Fetch historical stock data with robust error handling"""
        stock_data = {}
        for ticker in self.stocks:
            try:
                stock = yf.Ticker(ticker)
                data = stock.history(period=period)
                
                # Check if data is sufficient
                if len(data) > 0:
                    stock_data[ticker] = data
                else:
                    print(f"Insufficient data for {ticker}")
            except Exception as e:
                print(f"Error fetching {ticker}: {e}")
        return stock_data

    def calculate_momentum(self, stock_data):
        """Calculate momentum with dynamic range handling"""
        momentum_scores = {}
        for ticker, data in stock_data.items():
            try:
                # Dynamic range selection based on available data
                total_days = len(data)
                
                # Define momentum calculation periods
                periods = [
                    min(30, total_days),    # 1-month or max available
                    min(90, total_days),    # 3-month or max available
                    min(180, total_days),   # 6-month or max available
                    min(365, total_days)    # 1-year or max available
                ]
                
                returns = [
                    data['Close'].iloc[-1] / data['Close'].iloc[-period] - 1 
                    for period in periods
                ]
                
                momentum_scores[ticker] = np.mean(returns)
            
            except IndexError:
                print(f"Skipping {ticker} due to insufficient data")
            except Exception as e:
                print(f"Error processing {ticker}: {e}")
        
        return sorted(momentum_scores.items(), key=lambda x: x[1], reverse=True)

    def select_top_momentum_stocks(self, momentum_scores):
        """Select top momentum stocks"""
        return [stock[0] for stock in momentum_scores[:self.portfolio_size]]



--- Momentum Analysis ---

Momentum Scores:
BHARTIARTL.NS: 26.86%
ICICIBANK.NS: 16.24%
INFY.NS: 16.01%
HDFCBANK.NS: 10.10%
SBIN.NS: 8.98%
TCS.NS: 2.32%
BAJFINANCE.NS: -0.73%
KOTAKBANK.NS: -1.51%
MARUTI.NS: -4.14%
RELIANCE.NS: -10.55%

Selected Portfolio Stocks: ['BHARTIARTL.NS', 'ICICIBANK.NS', 'INFY.NS', 'HDFCBANK.NS', 'SBIN.NS']


In [ ]:

def main():
    # Indian Stock Market Example
    indian_stocks = [
        'INFY.NS',   # Infosys
        'TCS.NS',    # Tata Consultancy Services
        'RELIANCE.NS', # Reliance Industries
        'HDFCBANK.NS', # HDFC Bank
        'ICICIBANK.NS', # ICICI Bank
        'BHARTIARTL.NS', # Bharti Airtel
        'SBIN.NS',   # State Bank of India
        'MARUTI.NS', # Maruti Suzuki
        'BAJFINANCE.NS', # Bajaj Finance
        'KOTAKBANK.NS'   # Kotak Mahindra Bank
    ]
    
    portfolio = MomentumPortfolioBuilder(indian_stocks, portfolio_size=5)
    
    # Fetch stock data
    stock_data = portfolio.fetch_stock_data()
    
    # Calculate momentum
    momentum_scores = portfolio.calculate_momentum(stock_data)
    
    # Select top stocks
    top_stocks = portfolio.select_top_momentum_stocks(momentum_scores)
    
    # Display results
    print("\n--- Momentum Analysis ---")
    print("\nMomentum Scores:")
    for stock, score in momentum_scores[:10]:
        print(f"{stock}: {score*100:.2f}%")
    
    print("\nSelected Portfolio Stocks:", top_stocks)

if __name__ == "__main__":
    main()